# Language Modeling with Keras

In [110]:
# Handles Importing all the necessary libraries
import itertools
import h5py
import numpy as np
import string
from keras.layers import Input, Embedding, merge, Flatten, Reshape, Lambda, LSTM, Dropout
from keras.layers.recurrent import SimpleRNN
import keras.backend as K
from keras.models import Model
import sklearn
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 15, 6

### Prepares the Data Set to be used

In [106]:
#imports a massive dataset of sentences found on wikipedia
with open('ignored_assets/lang-train-data-angelas-ashes.txt') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line,
# also strips the punctuation
sentences = [x.strip().translate(None, string.punctuation) for x in content] 

lemma = lambda x: x.strip().lower().split(' ')
sentences_lemmatized = [lemma(sentence) for sentence in sentences]
words = set(itertools.chain(*sentences_lemmatized))
# set(['boy', 'fed', 'ate', 'cat', 'kicked', 'hat'])

# dictionaries for converting words to integers and vice versa
word2idx = dict((v, i) for i, v in enumerate(words))
idx2word = list(words)

# convert the sentences a numpy array
to_idx = lambda x: [word2idx[word] for word in x]
sentences_idx = [to_idx(sentence) for sentence in sentences_lemmatized]
# Sets the maximum word length of each sentence
max_len = 40
# a list of all the indices I remove that are longer than max_len
indices_removed = []
# If the sentence is too long, good by
for i in range(len(sentences_idx)-1, -1, -1):
    if len(sentences_idx[i]) > max_len:
        indices_removed = indices_removed + [i]
        sentences_idx.pop(i)
    elif len(sentences_idx[i]) < max_len:
        sentences_idx[i] = [0] * (max_len - len(sentences_idx[i])) + sentences_idx[i]
print "Indexes Removed: {}".format(len(indices_removed))
print "Number of Sentences: {}".format(len(sentences_idx))
print "Vocabulary: {}".format(len(word2idx))
sentences_array = np.asarray(sentences_idx, dtype='int32')

# Prepares the datasets as input and output
dataX = np.array(sentences_array[:len(sentences_array)-1]).copy()
dataY = np.array(sentences_array[1:]).copy()

# scales the output
scaler = MinMaxScaler(feature_range=(0, 1))
dataY = scaler.fit_transform(orig_dataY)
# reshapes the dataY
new_dataY = dataY.reshape((1,) + dataY.shape)


Indexes Removed: 323
Number of Sentences: 7441
Vocabulary: 6463


## Create and Train The Model

In [89]:
n_words = len(words)
n_embed_dims = 150

# put together a model to predict
input_sentence = Input(shape=(max_len,), dtype='int32')
input_embedding = Embedding(n_words, n_embed_dims)(input_sentence)
output = SimpleRNN(max_len)(input_embedding)

model = Model(inputs=[input_sentence], outputs=[output])
model.compile(optimizer='rmsprop', loss='mse')

# fit the model to predict what color each person is
model.fit(dataX, dataY, epochs=2, verbose=3)
embeddings = model.layers[1].get_weights()

# print out the embedding vector associated with each word
for i in range(2):
    print('{}: {}'.format(idx2word[i], embeddings[0][i]))
    
print "Evaluation of the Model"
print model.evaluate(dataX, dataY)
print "Saving the Weights"
model.save_weights('model.h5')

Epoch 1/2
Epoch 2/2
: [-0.01661432 -0.00696552 -0.0105041  -0.02706422 -0.032588    0.05675888
 -0.03586446 -0.04565528 -0.05752029  0.05073267 -0.01106296  0.03993874
 -0.01543578 -0.03887717  0.0067728  -0.00853418  0.00094086  0.02676368
  0.03613407  0.03612905 -0.03502238  0.03959559 -0.0372909   0.03599412
 -0.0669523   0.04242242  0.00354336  0.02336046  0.04605285  0.0130783
  0.00475045 -0.0038187  -0.01047738 -0.01872551 -0.02171952  0.04167085
  0.01607439  0.03657294  0.03171887  0.06265767  0.06442416  0.04387547
 -0.06585166  0.00996963 -0.04687055 -0.04212037  0.02033991  0.04103671
 -0.02860193  0.03982403  0.01655041  0.0280925  -0.02766712 -0.02552239
  0.06964874  0.03138467  0.02922536  0.01836007  0.03308022 -0.01765107
 -0.03590905 -0.0274533  -0.05558426  0.00449759 -0.00061701 -0.06623379
 -0.00473527 -0.00820161  0.04665337 -0.00797568 -0.00082443  0.04642951
  0.0338419  -0.02822362  0.0490189   0.04460569  0.01967261  0.02357109
  0.04668439 -0.02029392 -0.02

#### Extra Training For The Model

In [109]:
model.fit(dataX, dataY, epochs=20, batch_size=64, verbose=3)
print "  {}".format(model.evaluate(dataX, dataY))

Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
7328/7440 [============================>.] - ETA: 0s  0.0244942809986


## Further Training the Model

In [28]:
model.fit(dataX, dataY, epochs=4, batch_size=64, verbose=3)
embeddings = model.layers[1].get_weights()

# print out the embedding vector associated with each word
for i in range(2):
    print('{}: {}'.format(idx2word[i], embeddings[0][i]))

print 'Evaluation: {}'.format(model.evaluate(dataX, dataY))
model.save_weights('model.h5')

Epoch 1/4
Epoch 2/4
Epoch 3/4
Epoch 4/4
: [ -7.52860755e-02  -4.54015285e-02  -4.58094329e-02   8.84827878e-03
  -8.48692900e-04  -3.17476653e-02  -1.13650709e-02   6.56244531e-02
   6.53840974e-03   6.65657148e-02  -2.31543127e-02  -3.21851298e-02
  -5.29217646e-02   1.71483925e-03   1.58685390e-02   1.47633646e-02
  -3.10055520e-02   2.67946278e-03  -4.41819243e-03   1.17825922e-02
   3.26833390e-02   6.82228524e-03  -2.40664147e-02   1.81059614e-02
   4.85693328e-02   4.03854847e-02   5.97351231e-02   2.43128533e-03
  -2.56353430e-02   1.97669603e-02  -8.07724521e-03   6.48374995e-03
  -6.76449202e-03   3.24921831e-02   8.01629853e-03   9.09802562e-04
  -1.93910580e-02  -2.17047911e-02  -4.88672033e-02  -4.92293611e-02
  -1.37611022e-02   4.78589945e-02   1.62587315e-02  -1.04369866e-02
  -3.82489036e-03  -7.70846556e-04   6.44029975e-02   6.63364481e-04
   3.35666910e-02  -5.23372591e-02   6.23806678e-02  -1.97675209e-02
  -1.34685580e-02  -2.47143134e-02  -3.12603428e-03   3.02405

### Attempt to test the model with some custom inputs

In [24]:
inp = [0] * (max_len - 3) + [word2idx['i'], word2idx['love'], word2idx['you']]
model.predict(np.array(inp).reshape(1, 100))

array([[-0.        ,  0.        , -0.        ,  0.        ,  0.00316837,
        -0.        ,  0.        ,  0.        , -0.        , -0.        ,
         0.08428954,  0.28899291,  0.        ,  0.        ,  0.        ,
        -0.31553316,  0.03101788,  0.        ,  0.        , -0.        ,
        -0.        ,  0.        ,  0.13992527, -0.11859532,  0.        ,
         0.        ,  0.        ,  0.01655224, -0.        ,  0.03247942,
        -0.        ,  0.01039564, -0.        ,  0.        ,  0.19620955,
         0.        ,  0.        ,  0.18380126,  0.12647456, -0.        ,
         0.        ,  0.29543972,  0.59864187,  0.29531297,  0.28910464,
        -0.        ,  0.65704238,  0.83109629,  0.65468234,  0.56219828,
        -0.        ,  0.46520701,  0.54404628,  1.        ,  0.70375729,
         0.84097189,  1.        ,  1.        ,  1.        , -0.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        , -0.        ,  1. 

## Exporting the Model
This is how you export the model into a json file in order to be imported later. Then you export the model's weights. Later on in other experiments, you could effectively 

In [26]:

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
 

## Importing the Model
This is how you import the model from a json file and the weights so that you don't need to train it every time.

In [ ]:

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")